## Section 9: Summary and Next Steps

### Test Results Summary

- ✅ **Basic analysis**: engine runs without LLM
- ✅ **Results display**: all output fields accessible
- ✅ **Context testing**: multiple contexts work correctly
- ✅ **Error handling**: invalid inputs caught properly
- ✅ **JSON export**: results saved to file
- ✅ **Sync wrapper**: synchronous version works

### Next Steps

1. **Production Integration**
   - Integrate `run_engine()` into your web API or application
   - Add appropriate error handling and logging

2. **Performance Optimization**
   - Monitor analysis time for different audio lengths
   - Consider caching or batching for multiple files

3. **LLM Testing**
   - Enable `use_llm=True` for more accurate semantic analysis
   - Budget 60-120 seconds per file with LLM enabled

4. **Multiple Audio Files**
   - Test with various audio formats (WAV, MP3, etc.)
   - Verify handling of different audio qualities

### Useful Links

- Main implementation: `src/engine.py`
- Runner implementation: `src/engine_runner.py`
- Documentation: `IMPLEMENTATION_GUIDE.md`
- Quick test script: `test_quick.py`

In [ ]:
if audio_bytes:
    print("Testing synchronous wrapper function...\n")
    
    try:
        print("🔄 Running run_engine_sync()...")
        sync_result = run_engine_sync(
            audio_bytes=audio_bytes,
            context="conversational",
            use_llm=False,
            filename=selected_audio_file.name
        )
        
        print(f"✓ Sync test completed")
        print(f"  Overall band: {sync_result['band_scores']['overall_band']}")
        print(f"  Transcript length: {len(sync_result['transcript'])} chars")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
else:
    print("❌ No audio loaded")

## Section 8: Synchronous Test (run_engine_sync)

Test the synchronous wrapper function which doesn't require async/await

In [ ]:
if analysis_result:
    # Create output directory
    output_dir = PROJECT_ROOT / "outputs" / "notebook_tests"
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = output_dir / f"engine_test_{timestamp}.json"
    
    # Save results
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(analysis_result, f, indent=2, ensure_ascii=False)
    
    print(f"✓ Results exported to: {output_file}")
    print(f"  File size: {output_file.stat().st_size / 1024:.2f} KB")
else:
    print("❌ No results to export")

## Section 7: Export Results to JSON

In [ ]:
print("Testing error handling scenarios...\n")

# Test 1: Empty audio bytes
print("Test 1: Empty audio bytes")
try:
    result = await run_engine(audio_bytes=b"", context="conversational")
    print("❌ Should have raised ValueError")
except ValueError as e:
    print(f"✓ Correctly caught ValueError: {str(e)}")
except Exception as e:
    print(f"✓ Caught exception: {type(e).__name__}: {str(e)}")

print()

# Test 2: Invalid audio data
print("Test 2: Invalid audio data")
try:
    result = await run_engine(
        audio_bytes=b"\x00\x01\x02\x03\x04\x05",
        context="conversational"
    )
    print("❌ Should have raised error for invalid audio")
except Exception as e:
    print(f"✓ Correctly caught error: {type(e).__name__}")

print()

# Test 3: Invalid context
print("Test 3: Invalid context")
if audio_bytes:
    try:
        result = await run_engine(
            audio_bytes=audio_bytes,
            context="invalid_context"  # Not a standard context
        )
        print("✓ Accepted invalid context (may be handled gracefully)")
    except Exception as e:
        print(f"✓ Caught error: {type(e).__name__}")
else:
    print("⊘ Skipped (no audio loaded)")

print("\n✓ Error handling tests complete")

## Section 6: Test Error Handling

In [ ]:
if audio_bytes:
    contexts = ["conversational", "narrative", "presentation"]
    context_results = {}
    
    print("Testing different speech contexts...\n")
    
    for ctx in contexts:
        try:
            print(f"🔄 Testing context: {ctx}...")
            result = await run_engine(
                audio_bytes=audio_bytes,
                context=ctx,
                use_llm=False
            )
            context_results[ctx] = result['band_scores']['overall_band']
            print(f"   Band score: {result['band_scores']['overall_band']}\n")
        except Exception as e:
            print(f"   ❌ Error: {str(e)}\n")
    
    # Compare results
    print("\n📊 CONTEXT COMPARISON:")
    print("-" * 40)
    for ctx, band in context_results.items():
        print(f"{ctx:20s}: {band}")
else:
    print("❌ No audio loaded")

## Section 5: Test with Different Contexts

In [ ]:
if analysis_result:
    print("="*70)
    print("ANALYSIS RESULTS")
    print("="*70)
    
    # 1. TRANSCRIPT
    print("\n📝 TRANSCRIPT:")
    transcript = analysis_result['transcript']
    print(f"{transcript}\n")
    
    # 2. BAND SCORES
    print(f"🎯 IELTS BAND SCORES:")
    overall = analysis_result['band_scores']['overall_band']
    print(f"   Overall Band: {overall}")
    
    print(f"\n   Criterion Bands:")
    for criterion, band in analysis_result['band_scores']['criterion_bands'].items():
        print(f"     • {criterion}: {band}")
    
    # 3. METADATA
    print(f"\n⏱️  METADATA:")
    meta = analysis_result['metadata']
    print(f"   Duration: {meta['audio_duration_sec']}s")
    print(f"   Speaking time: {meta['speaking_time_sec']}s")
    print(f"   Total words: {meta['total_words_transcribed']}")
    print(f"   Content words: {meta['content_word_count']}")
    
    # 4. STATISTICS
    print(f"\n📊 STATISTICS:")
    stats = analysis_result['statistics']
    print(f"   Words transcribed: {stats['total_words_transcribed']}")
    print(f"   Content words: {stats['content_words']}")
    print(f"   Filler words: {stats['filler_words_detected']}")
    print(f"   Filler %: {stats['filler_percentage']}%")
    print(f"   Monotone: {'Yes' if stats['is_monotone'] else 'No'}")
    
    # 5. FLUENCY ANALYSIS
    fluency = analysis_result['fluency_analysis']
    if fluency:
        print(f"\n⚡ FLUENCY METRICS:")
        for key, value in fluency.items():
            if isinstance(value, float):
                print(f"   • {key}: {value:.3f}")
            else:
                print(f"   • {key}: {value}")
    
    # 6. PRONUNCIATION
    print(f"\n🔊 PRONUNCIATION:")
    pronun = analysis_result['pronunciation']
    intel = pronun['intelligibility']
    print(f"   Mean word confidence: {intel['mean_word_confidence']:.3f}")
    print(f"   Low confidence ratio: {intel['low_confidence_ratio']:.3f}")
    print(f"   Monotone detected: {pronun['prosody']['monotone_detected']}")
    
    # 7. FEEDBACK
    if analysis_result['band_scores']['feedback']:
        print(f"\n💬 FEEDBACK:")
        feedback = analysis_result['band_scores']['feedback']
        for criterion, text in feedback.items():
            print(f"\n   {criterion}:")
            print(f"   {text}")
    
    print(f"\n{'='*70}")
else:
    print("❌ No analysis results to display")

## Section 4: Display Analysis Results

In [ ]:
if audio_bytes:
    print("🚀 Starting analysis (LLM disabled for speed)...\n")
    
    try:
        # Run analysis
        result = await run_engine(
            audio_bytes=audio_bytes,
            context="conversational",
            device="cpu",
            use_llm=False,  # Faster for testing
            filename=selected_audio_file.name
        )
        
        # Store result for later
        analysis_result = result
        
        print("\n✓ Analysis completed successfully!\n")
        
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        analysis_result = None
        import traceback
        traceback.print_exc()
else:
    print("❌ No audio loaded")

## Section 3: Test run_engine() - Basic (No LLM)

In [ ]:
# Find and load sample audio
data_dir = PROJECT_ROOT / "data" / "ielts_part_2"

if not data_dir.exists():
    print(f"❌ Data directory not found: {data_dir}")
else:
    audio_files = sorted(data_dir.glob("*.wav"))
    print(f"📁 Found {len(audio_files)} audio files in {data_dir}")
    
    if audio_files:
        # Load first audio file
        audio_file = audio_files[0]
        print(f"\n📂 Selected: {audio_file.name}")
        print(f"   Size: {audio_file.stat().st_size / 1024 / 1024:.2f} MB")
        
        # Read audio bytes
        with open(audio_file, "rb") as f:
            audio_bytes = f.read()
        
        print(f"   Loaded: {len(audio_bytes)} bytes")
        
        # Store for later use
        selected_audio_file = audio_file
        print("\n✓ Audio loaded successfully")
    else:
        print("❌ No audio files found")
        selected_audio_file = None
        audio_bytes = None

## Section 2: Load Sample Audio

In [ ]:
import asyncio
import sys
import json
from pathlib import Path
from datetime import datetime

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Import engine runner
from src.engine_runner import run_engine, run_engine_sync
from src.logging_config import setup_logging

# Setup logging
logger = setup_logging(level="INFO")

print("✓ Imports successful")
print(f"Project root: {PROJECT_ROOT}")

## Section 1: Import Required Libraries

# Engine Runner Testing Notebook

Comprehensive testing notebook for `src/engine_runner.py`

This notebook demonstrates:
- Loading audio bytes from files
- Running the `run_engine()` function
- Processing and displaying results
- Testing different configurations